I am working on this new logs anaylsis notebook. Hopefully it will be able to generate logs, training graph metrics, etc. with a few click of a button.  

-Nathan Liang

In [ ]:
import json
import pandas as pd

In [ ]:
run_env = 'run.env'
with open(run_env, 'r') as f:
    for line in f.readlines():
        if "DR_LOCAL_S3_MODEL_PREFIX=" in line:
            spl_pnt = '='
            MODEL_PREFIX = line.partition(spl_pnt)[2]
            MODEL_PREFIX = MODEL_PREFIX.replace('\n','')
            
MODEL_PREFIX

'rl-deepracer-1'

In [ ]:
metrics1_fname = 'data/minio/bucket/%s/metrics/TrainingMetrics.json' %MODEL_PREFIX
metrics2_fname = 'data/minio/bucket/%s/metrics/TrainingMetrics_1.json' %MODEL_PREFIX

In [ ]:
def json_to_list(file):
    file = file.replace("{","[")
    file = file.replace("}","]")
    #file = file.replace(', ', '], [')
    file = file.replace('"', '')
    file = file.replace('[metrics: ',"")
    file = file.replace(', version: 2.0, best_model_metric: progress]',"")
    
    file = file.replace('reward_score: ', '"')
    file = file.replace(', metric_time: ', '","')
    file = file.replace(', start_time: ', '","')
    file = file.replace(', elapsed_time_in_milliseconds: ', '","')
    file = file.replace(', episode: ', '","')
    file = file.replace(', trial: ', '","')
    file = file.replace(', phase: ', '","')
    file = file.replace(', completion_percentage: ', '","')
    file = file.replace(', episode_status: ', '","')
    file = file.replace(']', '"]')
    file = file.replace('"]"]', '"]]')
    #print(file)
    return json.loads(file)

In [ ]:
with open(metrics1_fname, 'r') as file1:
    for line in file1:
        metrics1 = json_to_list(line)
with open(metrics2_fname, 'r') as file2:
    for line in file2:
        metrics2 = json_to_list(line)
print(type(metrics1))

<class 'list'>


In [ ]:
def parse_data(data_list):
    training = []
    evaluation = []
    for d in range(len(data_list)):
        if data_list[d][6] == 'training':
            episode = float(data_list[d][4])
            reward = float(data_list[d][0])
            progress = float(data_list[d][7])
            status = float(data_list[d][8])
            elapsed_time = float(data_list[d][3])
            training.append([episode, reward, status])
        elif data_list[d][6] == 'evaluation':
            episode = float(data_list[d][4])
            reward = float(data_list[d][0])
            progress = float(data_list[d][7])
            elapsed_time = float(data_list[d][3])
            status = float(data_list[d][8])
            evaluation.append([episode, reward, status])
            
    return [training, evaluation]

In [ ]:
training_data1, evaluation_data1 = parse_data(metrics1)
training_data2, evaluation_data2 = parse_data(metrics2)

In [ ]:
header = ['episode', 'reward', 'progress', 'elapsed_time', 'status']
sim_df1 = pd.DataFrame(training_data1, columns=header)
sim_df2 = pd.DataFrame(training_data2, columns=header)
sim_df1

'2'

In [ ]:
def plot_reward_graph(df_slice_iterations=sim_df1, df_slice_entropy=trn_df_entropy):
    font_size=16
    if(len(df_slice_iterations)>0):
        fig = plt.figure(figsize=(16, 10))
        ax = plt.gca()  # gca stands for 'get current axis'
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))  # force integer labels on x-axis
        ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

        df_slice_entropy.plot(kind='line',x='iteration',y='avg_entropy',label='',linewidth=1,color='red',alpha=0.3,fontsize=font_size,ax=ax)
        df_slice_entropy.plot(kind='line',linestyle='solid',x='iteration',y='avg_entropy_SMA3',linewidth=1.5,color='red',fontsize=font_size,ax=ax)

        df_slice_iterations.plot(kind='line',x='iteration',y='total_rewards_normalized',label='',linewidth=1,color='green',alpha=0.3,fontsize=font_size,ax=ax2)
        df_slice_iterations.plot(kind='line',linestyle='solid',x='iteration',y='total_rewards_normalized_SMA3',linewidth=1.5,color='green',fontsize=font_size,ax=ax2)

        df_slice_iterations.plot(kind='line',x='iteration',y='avg_progress',label='',linewidth=1,color='blue',alpha=0.3,fontsize=font_size,ax=ax2)
        df_slice_iterations.plot(kind='line',linestyle='dashdot',x='iteration',y='avg_progress_SMA3',linewidth=3,color='blue',fontsize=font_size,ax=ax2)

        ax.legend().remove()
        ax.set_xlabel(ax.get_xlabel(), fontsize=font_size)
        ax.set_ylabel('Entropy', fontsize=font_size)

        ax2.legend().remove()
        ax2.set_ylabel('Normalized Rewards / Progress (%) / Completed Laps (%)', fontsize=font_size)

        start_time = df_slice_iterations["start_time"].min()
        end_time = df_slice_iterations["end_time"].max()
        elapsed_time = end_time - start_time
        elapsed_time_hrs = elapsed_time / 3600
        plt.plot([], [], ' ', label='Iterations: %d' % df_slice_iterations["iteration"].max())
        plt.plot([], [], ' ', label='Elapsed Time: %0.2fhrs' % elapsed_time_hrs)

        fig.legend(loc="lower center", borderaxespad=0.1, ncol=4, fontsize=14, title="Legend")
        plt.subplots_adjust(bottom=0.15)

        max_rewards_iter = df_slice_iterations['total_rewards_normalized_SMA3'].idxmax()
        if (max_rewards_iter >= 0):
            xmax_rewards = df_slice_iterations['iteration'][max_rewards_iter]
            ymax_rewards = df_slice_iterations['total_rewards_normalized_SMA3'].max()
            plt.axvline(x=xmax_rewards,linestyle='dotted',linewidth=0.75,color='black')
            plt.axhline(y=ymax_rewards,linestyle='dotted',linewidth=0.75,color='black',alpha=0.3)
            plt.gca().text(xmax_rewards*0.995, ymax_rewards*1.005, 'Max Rewards @ %d' % xmax_rewards, ha='right', va='bottom', size=font_size)

        max_progress_iter = df_slice_iterations['avg_progress_SMA3'].idxmax()
        if (max_progress_iter >= 0):
            xmax_progress = df_slice_iterations['iteration'][max_progress_iter]
            ymax_progress = df_slice_iterations['avg_progress_SMA3'].max()
            plt.axvline(x=xmax_progress,linestyle='dotted',linewidth=0.75,color='black')
            plt.axhline(y=ymax_progress,linestyle='dotted',linewidth=0.75,color='black',alpha=0.3)
            plt.gca().text(xmax_progress*0.995, ymax_progress*1.005, 'Max Progress @ %d' % xmax_progress, ha='right', va='bottom', size=font_size)

        plt.yticks(np.arange(0, 105, step=10))
        plt.show()
        if (max_rewards_iter >= 0):
            print ("Iteration with Max Total Rewards (SMA3): %d" % (xmax_rewards))
        print ("Elapsed Time: %0.2fs (%0.2fhrs)" % (elapsed_time, elapsed_time_hrs))


plot_reward_graph()